In [ ]:
import pandas as pd
import numpy as np
import cv2
import numpy as np
from PIL import Image
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import os
import shutil

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# 암이 전이 안 됨 : 514, 암이 전이가 됨 : 486

In [ ]:
train['img_path'][0]

In [ ]:
for img_path in train['img_path']: # test도 똑같이 수행
    img_name = img_path.split('/')[-1] # 이미지 이름
    new_img = []
    new_img2 = []

    # 가로 방향으로 불필요한 흰색 배경 제거

    img = cv2.imread(img_path)[200:-200,200:-200,] # 이미지에 줄 같은 게 있어서 이를 제거하기 위해 200px씩 상하좌우로 제거
    img2 = cv2.cvtColor(img[:], cv2.COLOR_BGR2GRAY) # 이미지를 흑백으로 전환 
    ret, binary = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY) # 이미지 이진화
    binary = cv2.cvtColor(binary, cv2.COLOR_BGR2GRAY) # (?,?,3) -> (?,?)
    rows_to_delete = [i for i in range(len(binary)) if set(binary[i]) == {255}] 
    # 만약 이진화 이미지에 임파선이 없고 모두 흰 배경인 index

    for i in range(3):
        new_img.append(np.delete(img[:,:,i], rows_to_delete, axis=0)) # RGB 채널별로 흰 배경을 제거
        # 컬러 이미지를 유지하기 위해서

    new_img = np.array(new_img).T.swapaxes(0, 1) # -> (3, 2603, 5741) -> (5741,2603,3) -> (2603, 5741, 3)


    # 세로 방향으로 불필요한 흰색 배경 제거

    img = new_img
    img2 = cv2.cvtColor(img[:], cv2.COLOR_BGR2GRAY) # 흑백 전환
    ret, binary = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY) # 이미지 이진화 
    binary = cv2.cvtColor(binary, cv2.COLOR_BGR2GRAY) # 이진화된 이미지 흑백 전환

    rows_to_delete = [i for i in range(len(binary.T)) if set(binary.T[i]) == {255}] # 흰색만 있는 영역의 index
    for i in range(3):
        new_img2.append(np.delete(img[:,:,i].T, rows_to_delete, axis=0)) # RGB 채널별로 흰 배경 제거

    new_img2 = np.array(new_img2).T
    new_img2 = cv2.resize(new_img2,(1000,1000)) # 이미지가 cnn이 학습할 수 있게 (1000,1000,3)으로 사이즈 조절

    denoised_image = cv2.fastNlMeansDenoisingColored(new_img2, None, 10, 10, 7, 21) 
    # 이미지에 존재하는 희미한 점 등을 제거

    res = Image.fromarray(denoised_image) # 넘파이 배열을 이미지로 변환 
    res.save(f'./test_img_color/{img_name}','PNG') # png파일로 해당 이미지 저장

In [ ]:
for i in range(2):
    for img_path in list(train[:][train['N_category'] == i]['img_path']):
        x = img_path.split('/')
        shutil.copy(img_path, f'.\\trainset\\{str(i)}\\' + x[-1])

# train 데이터셋을 만들기 위해 trainset에 클래스별로 폴더 생성 후 이미지를 복제

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  'D:\cancer_dacon\\trainset',
  validation_split=0.1,
  subset="training",
  seed=42,
  image_size=(1000,1000))

# 기존 폴더에서 train데이터셋 생성

In [ ]:
from tensorflow.keras import Input

model = models.Sequential([
    layers.Conv2D(64, (3, 3), activation='relu', input_shape=(1000, 1000, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.GlobalAveragePooling2D(),
    layers.Flatten()
])

# CNN 모델 생성, 1차원 배열에서 최종적으로 이미지의 특성을 추출

In [ ]:
train['img_path'] = train['img_path'].str.replace('train_imgs','train_img_color')
test['img_path'] = test['img_path'].str.replace('test_imgs','test_img_color')

# 새롭게 전처리한 이미지가 있는 폴더를 대상으로 수행

In [ ]:
train_img_feature = []

for img_name in train['img_path']:
    img = cv2.imread(img_name)
    img = np.expand_dims(img, axis=0)  # 배치 차원 추가 (1, 500, 500, 3)

    f = model.predict(img)
    train_img_feature.append(f[0]) # model이 반환한 이미지의 특성을 리스트에 저장

In [ ]:
test_img_feature = []

for img_name in test['img_path']:
    img = cv2.imread(img_name)
    img = np.expand_dims(img, axis=0)  # 배치 차원 추가 (1, 500, 500, 3)

    f = model.predict(img)
    test_img_feature.append(f[0]) # model이 반환한 이미지의 특성을 리스트에 저장

In [ ]:
for col in train.columns:
    if train[col].isnull().sum() >= 700:
        train = train.drop(col,axis=1)
        test = test.drop(col,axis=1)

# 결측치가 70%를 넘어가면 해당 컬럼 제거

In [ ]:
for col in train.columns:
    if train[col].isnull().sum() >= 1 :
        train[col] = train[col].fillna(train[col].mode()[0])
        test[col] = test[col].fillna(test[col].mode()[0])

# 결측치의 경우 각 컬럼의 최빈값으로 채움

In [ ]:
train['histology_grade'] = train[['HG_score_1','HG_score_2','HG_score_3']].sum(axis=1)
test['histology_grade'] = test[['HG_score_1','HG_score_2','HG_score_3']].sum(axis=1)

train['histology_grade'] = np.where(train['histology_grade']<=5,1,
                           np.where(train['histology_grade']<=7,2,3))
test['histology_grade'] = np.where(test['histology_grade']<=5,1,
                           np.where(test['histology_grade']<=7,2,3))

# 세 점수를 더하면 조직학적으로 악성정도를 등급(1,2,3)을 매길 수 있음

In [ ]:
train['surgery_year'] = pd.to_datetime(train['수술연월일']).dt.year.astype('int64')
train['surgery_month'] = pd.to_datetime(train['수술연월일']).dt.month.astype('int64')
train['surgery_date'] = pd.to_datetime(train['수술연월일']).dt.day.astype('int64')

test['surgery_year'] = pd.to_datetime(test['수술연월일']).dt.year.astype('int64')
test['surgery_month'] = pd.to_datetime(test['수술연월일']).dt.month.astype('int64')
test['surgery_date'] = pd.to_datetime(test['수술연월일']).dt.day.astype('int64')

# 수술 연월일을 각각 분리하여 새로운 컬럼 생성

In [ ]:
down_bad_col = ['NG', 'HG', 'HG_score_1', 'HG_score_2', 'HG_score_3',
                'DCIS_or_LCIS_여부', 'T_category', 'ER', 'ER_Allred_score', 'PR',
                'PR_Allred_score', 'KI-67_LI_percent', 'HER2', 'HER2_IHC']

for col in down_bad_col:
    train[col] = np.where(train[col]==4,0,train[col])
    test[col] = np.where(test[col]==4,0,test[col])

# 해당 컬럼에서 4는 증상이 미미하여 거의 없다고 판별하기에 0과 동급으로 치환함

In [ ]:
train['serious'] = train[down_bad_col].sum(axis=1)
test['serious'] = test[down_bad_col].sum(axis=1)

# 숫자가 커질수록 안 좋은 컬럼들의 합계를 구함
# 숫자가 클수록 예측이 1에 가까울 것이라고 가정하여 해당 전처리 시도

In [ ]:
train = train.drop(['ID','수술연월일','mask_path','img_path'],axis=1)
test = test.drop(['ID','수술연월일','img_path'],axis=1)

In [ ]:
train['img_feature'] = train_img_feature
test['img_feature'] = test_img_feature

# 위에서 구한 이미지의 특성을 데이터셋에 추가

In [ ]:
from sklearn.decomposition import PCA

# 리스트 데이터를 개별 컬럼으로 변환
img_features_train = np.stack(train['img_feature'].values)
img_features_test = np.stack(test['img_feature'].values)

# PCA 차원 축소 -> 128개는 너무 많기 때문
pca = PCA(n_components=5) # 5개로 0.9904% 설명 가능
train_pca = pca.fit_transform(img_features_train)
test_pca = pca.transform(img_features_test)

# PCA 결과를 데이터프레임으로 변환
train_pca_df = pd.DataFrame(train_pca, columns=[f'pca_feature_{i}' for i in range(5)])
test_pca_df = pd.DataFrame(test_pca, columns=[f'pca_feature_{i}' for i in range(5)])

# 기존 데이터프레임에서 img_feature 제거 후 병합
train = train.drop(columns=['img_feature'])
train = pd.concat([train, train_pca_df], axis=1)

test = test.drop(columns=['img_feature'])
test = pd.concat([test, test_pca_df], axis=1)

In [ ]:
xtrain = train.drop('N_category',axis=1)
ytrain = train['N_category']

In [ ]:
from sklearn.model_selection import train_test_split

xtrain2,xval,ytrain2,yval = train_test_split(xtrain,ytrain,random_state=42, test_size=0.2)
xtrain2.shape, xval.shape

In [ ]:
from lightgbm import LGBMClassifier

model = LGBMClassifier(random_state=42)
model.fit(xtrain2,ytrain2)

In [ ]:
pred1 = model.predict(xtrain2)
pred2 = model.predict(xval)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

print(accuracy_score(ytrain2,pred1))
print(precision_score(ytrain2,pred1))
print(recall_score(ytrain2,pred1))
print(f1_score(ytrain2,pred1))
print()
print(accuracy_score(yval,pred2))
print(precision_score(yval,pred2))
print(recall_score(yval,pred2))
print(f1_score(yval,pred2))

In [ ]:
pr = model.predict(test)

In [ ]:
submit = pd.read_csv('./sample_submission.csv')
submit['N_category'] = pr
submit.to_csv('./submit.csv', index=False)